In [1]:
import dis

def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
    print('Blastoff!')

# Disassemble the function
dis.dis(countdown)


  3           0 RESUME                   0

  4           2 LOAD_FAST                0 (n)
              4 LOAD_CONST               1 (0)
              6 COMPARE_OP               4 (>)
             12 POP_JUMP_FORWARD_IF_FALSE    27 (to 68)

  5     >>   14 LOAD_GLOBAL              1 (NULL + print)
             26 LOAD_CONST               2 ('T-minus')
             28 LOAD_FAST                0 (n)
             30 PRECALL                  2
             34 CALL                     2
             44 POP_TOP

  6          46 LOAD_FAST                0 (n)
             48 LOAD_CONST               3 (1)
             50 BINARY_OP               23 (-=)
             54 STORE_FAST               0 (n)

  4          56 LOAD_FAST                0 (n)
             58 LOAD_CONST               1 (0)
             60 COMPARE_OP               4 (>)
             66 POP_JUMP_BACKWARD_IF_TRUE    27 (to 14)

  7     >>   68 LOAD_GLOBAL              1 (NULL + print)
             80 LOAD_CONST               

In [2]:
# Access the bytecode
bytecode = countdown.__code__.co_code
print(bytecode)


b'\x97\x00|\x00d\x01k\x04\x00\x00\x00\x00r\x1bt\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00d\x02|\x00\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00|\x00d\x03z\x17\x00\x00}\x00|\x00d\x01k\x04\x00\x00\x00\x00\xb0\x1bt\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00d\x04\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00d\x00S\x00'


In [3]:
import opcode

# Extract first opcode
c = countdown.__code__.co_code
print(opcode.opname[c[0]])  # First opcode
print(opcode.opname[c[3]])  # Second opcode


RESUME
CACHE


In [4]:
import opcode

def generate_opcodes(codebytes):
    extended_arg = 0
    i = 0
    n = len(codebytes)

    while i < n:
        op = codebytes[i]
        i += 1
        if op >= opcode.HAVE_ARGUMENT:
            oparg = codebytes[i] + codebytes[i+1]*256 + extended_arg
            extended_arg = 0
            i += 2
            if op == opcode.EXTENDED_ARG:
                extended_arg = oparg * 65536
                continue
        else:
            oparg = None

        yield (op, oparg)

# Use the function to parse countdown's bytecode
for op, oparg in generate_opcodes(countdown.__code__.co_code):
    print(op, opcode.opname[op], oparg)


151 RESUME 31744
0 CACHE None
100 LOAD_CONST 27393
4 <4> None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
114 POP_JUMP_FORWARD_IF_FALSE 29723
1 POP_TOP None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
100 LOAD_CONST 31746
0 CACHE None
166 PRECALL 2
0 CACHE None
171 CALL 2
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
1 POP_TOP None
0 CACHE None
124 LOAD_FAST 25600
3 <3> None
122 BINARY_OP 23
0 CACHE None
125 STORE_FAST 31744
0 CACHE None
100 LOAD_CONST 27393
4 <4> None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
176 POP_JUMP_BACKWARD_IF_TRUE 29723
1 POP_TOP None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
100 LOAD_CONST 42500
1 POP_TOP None
0 CACHE None
0 CACHE None
171 CALL 1
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 CACHE None
0 C

In [13]:
import types
import dis

def add(x, y):
    return x + y

# Print the disassembly of the original function to understand the bytecode
print("Original function disassembly:")
dis.dis(add)

# Get the code object
c = add.__code__

# Print original bytecode
print("\nOriginal bytecode:", c.co_code)

# Create a new bytecode sequence
# In Python 3.x, the bytecode for a simple add and return should be:
# LOAD_FAST(124) for x
# LOAD_FAST(124) for y
# BINARY_ADD(23)
# RETURN_VALUE(83)
newbytecode = b'|\x00|\x01\x17S'

# Create a new code object with all necessary arguments
nc = types.CodeType(
    c.co_argcount,
    c.co_posonlyargcount,
    c.co_kwonlyargcount,
    c.co_nlocals,
    c.co_stacksize,
    c.co_flags,
    newbytecode,
    c.co_consts,
    c.co_names,
    c.co_varnames,
    c.co_filename,
    c.co_name,
    c.co_firstlineno,
    c.co_lnotab,
    c.co_freevars,
    c.co_cellvars
)

# Replace function's code object
add.__code__ = nc

# Test the modified function
print("\nModified function output:", add(2, 3))  # Should print 5

# Verify the new bytecode
print("\nModified function disassembly:")
dis.dis(add)

Original function disassembly:
  4           0 RESUME                   0

  5           2 LOAD_FAST                0 (x)
              4 LOAD_FAST                1 (y)
              6 BINARY_OP                0 (+)
             10 RETURN_VALUE

Original bytecode: b'\x97\x00|\x00|\x01z\x00\x00\x00S\x00'


TypeError: code() argument 13 must be str, not int